In [ ]:
# -*- coding: utf-8 -*-
# 라이브러리 설치 확인 
import subprocess
import sys
from pathlib import Path
import pkg_resources  # 패키지 확인용

def install_requirements():
    """requirements.txt 에 있는 패키지를 확인 후 없으면 설치"""
    print("Python3.9 기준으로 제작되었습니다.")
    try:
        base_dir = Path(__file__).resolve().parent   # 일반 실행
    except NameError:
        base_dir = Path.cwd()                        # Jupyter 실행
    
    req_file = base_dir / "requirements.txt"
    if not req_file.exists():
        print("[오류] requirements.txt 파일이 없습니다.")
        return

    with open(req_file, "r", encoding="utf-8") as f:
        for line in f:
            package = line.strip()
            if not package or package.startswith("#"):
                continue

            pkg_name = package.split("==")[0]  # 버전 제외 이름만
            try:
                pkg_resources.get_distribution(pkg_name)
                print(f"[확인 완료] {package} 이미 설치됨 ✅")
            except pkg_resources.DistributionNotFound:
                print(f"[설치 필요] {package} 라이브러리가 없습니다. 설치를 진행합니다...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 실행 전에 라이브러리 확인 & 설치
install_requirements()

import sys
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from collections import deque
import time
from pathlib import Path

# PyQt5 관련 모듈
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QTextEdit, QHBoxLayout, QVBoxLayout,
    QPushButton, QDialog, QTextBrowser, QShortcut, QPlainTextEdit, QLineEdit,
    QComboBox, QCheckBox, QFrame
)
from PyQt5.QtGui import QImage, QPixmap, QKeySequence, QIcon
from PyQt5.QtCore import QThread, pyqtSignal, Qt, QPoint

# ======================= PART 01. 수어 인식 모델 및 데이터 처리 로직 =======================
# (이 부분의 코드는 변경되지 않았습니다.)
consonant_labels = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
vowel_labels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ','ㅣ','ㅚ','ㅟ','ㅢ']
command_labels = ['shift', 'space', 'b_space', 'end']
labels = consonant_labels + vowel_labels + command_labels
font_path = "C:/Windows/Fonts/gulim.ttc"
dataset_file = 'combine_4.csv'
first_spelling = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
second_spelling = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
last_spelling = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
start_kr = 44032
end_kr = 55203
def is_hangul(char): return start_kr <= ord(char) <= end_kr
def decompose(char):
    if not is_hangul(char): return None, None, None
    code = ord(char) - start_kr; last_index = code % 28; code //= 28; second_index = code % 21; first_index = code // 21
    return first_spelling[first_index], second_spelling[second_index], last_spelling[last_index]
def compose(first, second, last=' '):
    try:
        first_index, second_index, last_index = first_spelling.index(first), second_spelling.index(second), last_spelling.index(last)
        return chr(start_kr + (first_index * 588) + (second_index * 28) + last_index)
    except (ValueError, IndexError): return None
class HangulAssembler:
    def __init__(self):
        self.full_text = ""; self.command_shift = False
        self.double_consonant_map = {'ㄱ': 'ㄲ', 'ㄷ': 'ㄸ', 'ㅂ': 'ㅃ', 'ㅅ': 'ㅆ', 'ㅈ': 'ㅉ'}
        self.complex_last_map = {('ㄱ', 'ㅅ'): 'ㄳ', ('ㄴ', 'ㅈ'): 'ㄵ', ('ㄴ', 'ㅎ'): 'ㄶ', ('ㄹ', 'ㄱ'): 'ㄺ', ('ㄹ', 'ㅁ'): 'ㄻ', ('ㄹ', 'ㅂ'): 'ㄼ', ('ㄹ', 'ㅅ'): 'ㄽ', ('ㄹ', 'ㅌ'): 'ㄾ', ('ㄹ', 'ㅍ'): 'ㄿ', ('ㄹ', 'ㅎ'): 'ㅀ', ('ㅂ', 'ㅅ'): 'ㅄ'}
        self.dipthong_map = {('ㅗ', 'ㅏ'): 'ㅘ', ('ㅗ', 'ㅐ'): 'ㅙ', ('ㅗ', 'ㅣ'): 'ㅚ', ('ㅜ', 'ㅓ'): 'ㅝ', ('ㅜ', 'ㅔ'): 'ㅞ', ('ㅜ', 'ㅣ'): 'ㅟ', ('ㅡ', 'ㅣ'): 'ㅢ'}
        self.last_decompose_map = {v: k for k, v in self.complex_last_map.items()}
    def add_char(self, char):
        if char in command_labels: self._process_command(char)
        elif char in consonant_labels: self._process_consonant(char)
        elif char in vowel_labels: self._process_vowel(char)
        return self.full_text
    def _process_command(self, char):
        if char == 'shift': self.command_shift = True
        elif char == 'space': self.full_text += " "
        elif char == 'b_space': self._process_backspace()
        elif char == 'end': pass
    def get_current_text_and_reset(self):
        text_to_send = self.full_text
        self.full_text = ""
        self.command_shift = False
        return text_to_send
    def _process_backspace(self):
        if not self.full_text: return
        last_char = self.full_text[-1]
        if not is_hangul(last_char): self.full_text = self.full_text[:-1]; return
        first, second, last = decompose(last_char)
        if last != ' ':
            if last in self.last_decompose_map: new_char = compose(first,second, self.last_decompose_map[last][0])
            else: new_char = compose(first, second)
            self.full_text = self.full_text[:-1] + new_char
        elif second: self.full_text = self.full_text[:-1] + first
        else: self.full_text = self.full_text[:-1]
    def _process_consonant(self, char):
        if self.command_shift and char in self.double_consonant_map: char = self.double_consonant_map[char]
        self.command_shift = False
        last_char = self.full_text[-1] if self.full_text else None
        if last_char and is_hangul(last_char):
            first, second, last = decompose(last_char)
            if last == ' ' and char in last_spelling: self.full_text = self.full_text[:-1] + compose(first, second, char); return
            elif last in last_spelling and (last, char) in self.complex_last_map: self.full_text = self.full_text[:-1] + compose(first, second, self.complex_last_map[(last, char)]); return
        self.full_text += char
    def _process_vowel(self, char):
        self.command_shift = False
        last_char = self.full_text[-1] if self.full_text else None
        if not last_char: self.full_text += char; return
        if last_char in first_spelling: self.full_text = self.full_text[:-1] + compose(last_char, char); return
        if is_hangul(last_char):
            first, second, last = decompose(last_char)
            if last != ' ':
                if last in self.last_decompose_map:
                    last_1, last_2 = self.last_decompose_map[last]; syllable_1 = compose(first, second, last_1); syllable_2 = compose(last_2, char)
                    self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                else: syllable_1 = compose(first, second); syllable_2 = compose(last, char); self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                return
            else:
                if (second, char) in self.dipthong_map: self.full_text = self.full_text[:-1] + compose(first, self.dipthong_map[(second, char)]); return
        self.full_text += char
mp_hands = mp.solutions.hands; mp_drawing = mp.solutions.drawing_utils

# <<< 변경된 부분: putText_korean 함수 >>>
def putText_korean(image, text, pos, font_path, font_size, color):
    # 텍스트 색상(검정), 배경색(흰색)을 고정값으로 설정
    text_color_rgb = (0, 0, 0)
    bg_color_rgb = (255, 255, 255)

    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)

    # 텍스트 크기를 계산하여 배경 사각형의 좌표를 정함 (Pillow 10.0.0 이상 권장)
    try:
        # textbbox는 텍스트의 정확한 바운딩 박스를 반환 (left, top, right, bottom)
        text_bbox = draw.textbbox(pos, text, font=font)
    except AttributeError:
        # 구버전 Pillow에서는 textsize 사용
        text_width, text_height = draw.textsize(text, font=font)
        text_bbox = (pos[0], pos[1], pos[0] + text_width, pos[1] + text_height)

    # 텍스트 주변에 여백(padding) 추가
    padding = 5
    rectangle_pos = [
        (text_bbox[0] - padding, text_bbox[1] - padding),
        (text_bbox[2] + padding, text_bbox[3] + padding)
    ]

    # 흰색 배경 사각형 그리기
    draw.rectangle(rectangle_pos, fill=bg_color_rgb)

    # 검은색 텍스트 그리기
    draw.text(pos, text, font=font, fill=text_color_rgb)

    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def calculate_angles(joint):
    v1=joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]; v2=joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]; v=v2-v1; v=v/np.linalg.norm(v,axis=1)[:,np.newaxis]
    return np.degrees(np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))).astype(np.float32)
def calculate_distances(joint):
    thumb_tip = joint[4]; other_tips = joint[[8, 12, 16, 20]]; distances = np.linalg.norm(other_tips - thumb_tip, axis=1)
    return distances.astype(np.float32)
def calculate_orientation_vectors(joint):
    v_direction=joint[9]-joint[0]; v_direction=np.zeros(3) if np.linalg.norm(v_direction)==0 else v_direction/np.linalg.norm(v_direction)
    v1,v2=joint[5]-joint[0],joint[17]-joint[0]; v_normal=np.cross(v1,v2); v_normal=np.zeros(3) if np.linalg.norm(v_normal)==0 else v_normal/np.linalg.norm(v_normal)
    return np.concatenate([v_direction, v_normal]).astype(np.float32)
def load_and_preprocess(dataset_file):
    print("데이터셋 로드 및 전처리 시작...")
    try:
        labels_str = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=0, encoding="UTF-8", dtype=str)
        landmarks_data = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=range(1, 127), encoding="UTF-8").astype(np.float32)
    except Exception as e:
        print(f"데이터 파일 로드 오류: {e}"); return None, None, None
    all_features = []
    for row in landmarks_data:
        lh_landmarks = row[:63].reshape(21, 3); rh_landmarks = row[63:].reshape(21, 3)
        lh_angles = calculate_angles(lh_landmarks) if np.any(lh_landmarks) else np.zeros(15, dtype=np.float32)
        rh_angles = calculate_angles(rh_landmarks) if np.any(rh_landmarks) else np.zeros(15, dtype=np.float32)
        lh_coords = (lh_landmarks[1:]-lh_landmarks[0]).flatten() if np.any(lh_landmarks) else np.zeros(60, dtype=np.float32)
        rh_coords = (rh_landmarks[1:]-rh_landmarks[0]).flatten() if np.any(rh_landmarks) else np.zeros(60, dtype=np.float32)
        lh_distances = calculate_distances(lh_landmarks) if np.any(lh_landmarks) else np.zeros(4, dtype=np.float32)
        rh_distances = calculate_distances(rh_landmarks) if np.any(rh_landmarks) else np.zeros(4, dtype=np.float32)
        lh_orientation = calculate_orientation_vectors(lh_landmarks) if np.any(lh_landmarks) else np.zeros(6, dtype=np.float32)
        rh_orientation = calculate_orientation_vectors(rh_landmarks) if np.any(rh_landmarks) else np.zeros(6, dtype=np.float32)
        features = np.concatenate([lh_angles, rh_angles, lh_coords, rh_coords, lh_distances, rh_distances, lh_orientation, rh_orientation])
        all_features.append(features)
    all_features = np.array(all_features, dtype=np.float32); encoder = LabelEncoder(); encoded_labels = encoder.fit_transform(labels_str); print("데이터 전처리 완료!")
    return all_features, encoded_labels, encoder
def train_model(dataset_file):
    X, y, encoder = load_and_preprocess(dataset_file)
    if X is None: return None, None
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    print("--- 랜덤 포레스트 모델 학습 시작 ---"); model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred) * 100
    print(f"\n--- 학습 완료 ---\n모델 테스트 정확도: {acc:.4f}%")
    f1 = f1_score(y_test, y_pred, average='micro')
    print(f"f1_score: {f1:.6f}")
    return model, encoder

# ======================= PART 02. PyQt5 GUI 및 영상 처리 스레드 =======================
class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    update_text_signal = pyqtSignal(str)

    def __init__(self, model, encoder):
        super().__init__()
        self._run_flag = True
        self.model = model
        self.encoder = encoder

        self.last_rec_time = 0.0
        self.rec_cool_time = 3.0
        self.display_duration = 3.0

        # 변경점 1) 지연 오픈: 바로 열지 않고 None으로 시작
        self.cap = None
        self._is_paused = False
        self.show_landmarks = True

        # 재연결/설정
        self.camera_index = 0
        self.req_width, self.req_height = 640, 480
        self.reopen_interval_sec = 1.0
        self.read_fail_sleep_sec = 0.3

    # ---- 외부 제어 ----
    def pause(self): self._is_paused = True
    def resume(self): self._is_paused = False

    def set_camera(self, index:int):
        """실행 중 카메라 인덱스 변경 -> 다음 루프에서 재오픈"""
        self.camera_index = index
        self.update_text_signal.emit(f"📷 카메라 변경: index={index}")
        self._safe_release()

    def stop(self):
        self._run_flag = False
        self._safe_release()
        self.wait()

    # ---- 내부 유틸 ----
    def _backend_for_os(self):
        import sys
        if sys.platform.startswith("win"):
            return cv2.CAP_DSHOW
        elif sys.platform.startswith("linux"):
            return cv2.CAP_V4L2
        else:
            return 0  # mac 등 기본

    def _try_open(self) -> bool:
        """열려있지 않으면 오픈 재시도"""
        backend = self._backend_for_os()
        try:
            self.cap = cv2.VideoCapture(self.camera_index, backend) if backend else cv2.VideoCapture(self.camera_index)
        except Exception as e:
            self.cap = None
            self.update_text_signal.emit(f"⚠️ 카메라 열기 예외: {e}")
            time.sleep(self.reopen_interval_sec)
            return False

        if not (self.cap and self.cap.isOpened()):
            self.update_text_signal.emit(f"🔌 카메라 미연결/점유 중… (index={self.camera_index})")
            self._safe_release()
            time.sleep(self.reopen_interval_sec)
            return False

        # 가능한 해상도 설정
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.req_width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.req_height)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        self.update_text_signal.emit(f"✅ 카메라 연결됨 (index={self.camera_index})")
        return True

    def _safe_release(self):
        try:
            if self.cap is not None:
                self.cap.release()
        except Exception:
            pass
        self.cap = None

    # ---- 메인 루프 ----
    def run(self):
        history = deque(maxlen=5)
        display_label = ""
        display_start_time = None

        # 변경점 2) cap.isOpened() 의존 제거, _run_flag 기준으로 루프
        with mp_hands.Hands(max_num_hands=2,
                            min_detection_confidence=0.5,
                            min_tracking_confidence=0.5) as hands:
            while self._run_flag:
                # 일시정지
                if self._is_paused:
                    time.sleep(0.01)
                    continue

                # 열려있지 않으면 재오픈 시도
                if self.cap is None or not self.cap.isOpened():
                    if not self._try_open():
                        # 실패하면 잠시 대기 후 계속 (핫플러그 대비)
                        time.sleep(self.reopen_interval_sec)
                        continue

                # 프레임 읽기
                ret, frame = self.cap.read()
                if not ret or frame is None:
                    self.update_text_signal.emit("❌ 프레임 수신 실패… 재연결")
                    self._safe_release()
                    time.sleep(self.read_fail_sleep_sec)
                    continue

                frame = cv2.flip(frame, 1)
                result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                current_time = time.time()
                guide_text = "손을 보여주세요"
                hands_present = False

                if result.multi_hand_landmarks:
                    hands_present = True
                    init_zeros = {
                        'angles': np.zeros(15),
                        'coords': np.zeros(60),
                        'distances': np.zeros(4),
                        'orientation': np.zeros(6)
                    }
                    lh_features, rh_features = init_zeros.copy(), init_zeros.copy()

                    for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
                        if self.show_landmarks:
                            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                        handedness = result.multi_handedness[i].classification[0].label
                        joint = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])

                        features = {
                            'angles': calculate_angles(joint),
                            'coords': (joint[1:] - joint[0]).flatten(),
                            'distances': calculate_distances(joint),
                            'orientation': calculate_orientation_vectors(joint)
                        }
                        if handedness == "Left":
                            lh_features = features
                        elif handedness == "Right":
                            rh_features = features

                    feature_vector = np.concatenate([
                        lh_features['angles'], rh_features['angles'],
                        lh_features['coords'], rh_features['coords'],
                        lh_features['distances'], rh_features['distances'],
                        lh_features['orientation'], rh_features['orientation']
                    ]).reshape(1, -1)

                    predicted_label = self.encoder.inverse_transform(self.model.predict(feature_vector))[0]
                    history.append(predicted_label)

                    if len(history) == 5 and len(set(history)) == 1:
                        if current_time - self.last_rec_time > self.rec_cool_time:
                            mapped_label = history[-1]
                            self.update_text_signal.emit(mapped_label)
                            display_label = mapped_label
                            display_start_time = current_time
                            self.last_rec_time = current_time
                            history.clear()

                # 안내/상태 텍스트
                if hands_present:
                    if display_start_time and ((current_time - display_start_time) < self.display_duration):
                        display_text = display_label
                    else:
                        display_text = "인식 중..."
                else:
                    display_text = guide_text

                # (검은색 텍스트 유지)
                frame = putText_korean(frame, display_text, (50, 420), font_path, 40, (0, 0, 0))

                # UI 갱신
                self.change_pixmap_signal.emit(frame)

                # 약간의 쉼
                time.sleep(0.001)

        # 종료 정리
        self._safe_release()


class HelpWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent); self.setWindowTitle("도움말"); self.resize(591, 642)
        lay, title, body = QVBoxLayout(self), QLabel("<b>수어 예시</b>"), QTextBrowser()
        img_path = Path(r"C:\Users\KCCISTC\Desktop\workspace\hand_img.png")
        if not img_path.exists(): body.setText("도움말 이미지 파일을 찾을 수 없습니다.")
        else: body.setHtml(f'<br><img src="{img_path.as_uri()}" width="550"><br>')
        lay.addWidget(title); lay.addWidget(body)

class SettingsWindow(QDialog):
    speed_changed = pyqtSignal(float)
    landmark_visibility_changed = pyqtSignal(bool)
    def __init__(self, current_speed, landmark_visible, parent=None):
        super().__init__(parent)
        self.setWindowTitle("설정"); self.resize(350, 250)
        self.info_label1 = QLabel("인식 속도 조절"); self.info_label2 = QLabel("숫자가 낮을수록 인식 간격이 짧아져 빨라집니다.")
        self.speed_combo = QComboBox(); self.apply_button = QPushButton("적용")
        self.landmark_checkbox = QCheckBox("랜드마크 표시"); self.landmark_checkbox.setChecked(landmark_visible)
        self.speed_options = {"매우 느림 (5.0초)": 5.0, "느림 (4.0초)": 4.0, "보통 (3.0초)": 3.0, "빠름 (2.0초)": 2.0, "매우 빠름 (1.0초)": 1.0}
        for text, value in self.speed_options.items(): self.speed_combo.addItem(text, value)
        for i, value in enumerate(self.speed_options.values()):
            if value == current_speed: self.speed_combo.setCurrentIndex(i); break
        separator = QFrame(); separator.setFrameShape(QFrame.HLine); separator.setFrameShadow(QFrame.Sunken)
        layout = QVBoxLayout(self); layout.addWidget(self.info_label1); layout.addWidget(self.info_label2)
        layout.addWidget(self.speed_combo); layout.addWidget(separator); layout.addWidget(self.landmark_checkbox)
        layout.addStretch(1); layout.addWidget(self.apply_button)
        self.apply_button.clicked.connect(self.apply_settings)
    def apply_settings(self):
        selected_speed = self.speed_combo.currentData(); self.speed_changed.emit(selected_speed)
        is_visible = self.landmark_checkbox.isChecked(); self.landmark_visibility_changed.emit(is_visible)
        self.accept()

class SignLanguageTranslatorApp(QWidget):
    def __init__(self, model, encoder):
        super().__init__()
        self.setWindowTitle("수어 번역 프로그램"); self.setWindowIcon(QIcon("세종머왕.png"))
        self.current_rec_speed = 3.0; self.is_paused = False; self.show_landmarks = True
        self.camera_view = QLabel(self); self.camera_view.setObjectName("cameraView"); self.camera_view.setMinimumSize(600, 480)
        self.pause_button = QPushButton("일시정지"); self.settings_button = QPushButton("설정"); self.help_button = QPushButton("도움말")
        self.log_box = QPlainTextEdit(self); self.log_box.setReadOnly(True)
        self.bottom_input = QLineEdit(self)
        style_sheet = """
            QWidget { background-color: #2E2E2E; color: #F0F0F0; font-family: "Malgun Gothic"; font-size: 11pt; }
            #cameraView { border: 2px solid #00A0A0; border-radius: 5px; }
            QPushButton { background-color: #555555; border: 1px solid #777777; padding: 5px; border-radius: 5px; }
            QPushButton:hover { background-color: #6A6A6A; } QPushButton:pressed { background-color: #4A4A4A; }
            QPlainTextEdit, QLineEdit { background-color: #3C3C3C; border: 1px solid #555555; border-radius: 5px; font-size: 12pt; padding: 5px; }
            QLineEdit { selection-background-color: #00A0A0; selection-color: white; }
            QComboBox { border: 1px solid #777777; border-radius: 3px; padding: 1px 18px 1px 3px; min-width: 6em; background-color: #555555;}
            QComboBox::drop-down { subcontrol-origin: padding; subcontrol-position: top right; width: 20px; border-left-width: 1px; border-left-color: #777777; border-left-style: solid; border-top-right-radius: 3px; border-bottom-right-radius: 3px; }
            QCheckBox::indicator { width: 15px; height: 15px; }
        """
        self.setStyleSheet(style_sheet)
        button_layout = QHBoxLayout(); button_layout.addStretch(1); button_layout.addWidget(self.pause_button); button_layout.addWidget(self.settings_button); button_layout.addWidget(self.help_button)
        right_pane_layout = QVBoxLayout(); right_pane_layout.addLayout(button_layout); right_pane_layout.addWidget(self.log_box, stretch=10); right_pane_layout.addWidget(self.bottom_input, stretch=1)
        right_widget = QWidget(); right_widget.setLayout(right_pane_layout); right_widget.setFixedWidth(350)
        main_layout = QHBoxLayout(self); main_layout.addWidget(self.camera_view, stretch=2); main_layout.addWidget(right_widget, stretch=1)
        self.setLayout(main_layout); self.resize(980, 520)
        self.pause_button.clicked.connect(self.toggle_pause_resume)
        self.help_button.clicked.connect(self.toggle_help_window)
        self.settings_button.clicked.connect(self.open_settings_window)
        self.bottom_input.returnPressed.connect(self.finalize_sentence)
        self.help_window, self.settings_window = None, None
        self.assembler = HangulAssembler()
        self.thread = VideoThread(model, encoder)
        self.thread.change_pixmap_signal.connect(self.update_image)
        self.thread.update_text_signal.connect(self.update_text)
        self.thread.start()
        self.quit_shortcut = QShortcut(QKeySequence('q'), self); self.quit_shortcut.activated.connect(self.close)
        self.help_shortcut = QShortcut(QKeySequence(Qt.Key_F1), self); self.help_shortcut.activated.connect(self.toggle_help_window)
    
    def toggle_pause_resume(self):
        self.is_paused = not self.is_paused
        if self.is_paused: self.thread.pause(); self.pause_button.setText("다시 시작")
        else: self.thread.resume(); self.pause_button.setText("일시정지")

    def open_settings_window(self):
        if self.settings_window is None or not self.settings_window.isVisible():
            self.settings_window = SettingsWindow(self.current_rec_speed, self.show_landmarks, self)
            self.settings_window.speed_changed.connect(self.update_recognition_speed)
            self.settings_window.landmark_visibility_changed.connect(self.update_landmark_visibility)
            self.settings_window.show()
        else: self.settings_window.activateWindow()

    def update_recognition_speed(self, new_speed):
        self.current_rec_speed = new_speed; self.thread.rec_cool_time = new_speed; self.thread.display_duration = new_speed
        print(f"인식 속도가 {new_speed}초로 변경되었습니다.")

    def update_landmark_visibility(self, is_visible):
        self.show_landmarks = is_visible; self.thread.show_landmarks = is_visible
        print(f"랜드마크 표시: {'ON' if is_visible else 'OFF'}")

    def finalize_sentence(self):
        text = self.bottom_input.text()
        if text: self.log_box.appendPlainText(text)
        self.bottom_input.clear(); self.assembler = HangulAssembler()
        
    def update_text(self, new_char):
        if new_char == 'end':
            text_to_send = self.assembler.get_current_text_and_reset()
            if text_to_send: self.log_box.appendPlainText(text_to_send)
            self.bottom_input.clear()
            return
        current_text = self.assembler.add_char(new_char)
        self.bottom_input.setText(current_text)
        
    def toggle_help_window(self):
        if self.help_window is None or not self.help_window.isVisible():
            self.help_window = HelpWindow(self); self.help_window.setModal(False)
            self.help_window.move(self.frameGeometry().topRight() + QPoint(10, 0)); self.help_window.show()
        else: self.help_window.close()
            
    def update_image(self, cv_img):
        qt_img = self.convert_cv_qt(cv_img); self.camera_view.setPixmap(qt_img)
        
    def convert_cv_qt(self, cv_img):
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB); h, w, ch = rgb_image.shape; p = QPixmap.fromImage(QImage(rgb_image.data, w, h, ch * w, QImage.Format_RGB888))
        return p.scaled(self.camera_view.width(), self.camera_view.height(), Qt.KeepAspectRatio)
        
    def closeEvent(self, event):
        if self.help_window and self.help_window.isVisible(): self.help_window.close()
        if self.settings_window and self.settings_window.isVisible(): self.settings_window.close()
        self.thread.stop(); event.accept(); cv2.destroyAllWindows(); cv2.waitKey(1)

# ======================= PART 03. 메인 실행 부분 =======================
if __name__ == "__main__":
    trained_model, label_encoder = train_model(dataset_file)
    if trained_model and label_encoder:
        app = QApplication(sys.argv)
        window = SignLanguageTranslatorApp(trained_model, label_encoder)
        window.show()
        sys.exit(app.exec_())
    else:
        print("모델 학습에 실패하여 프로그램을 종료합니다.")